In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
gender_submission = pd.read_csv('/kaggle/input/titanic/gender_submission.csv')

In [3]:
gender_submission

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [4]:
train = pd.read_csv('/kaggle/input/titanic/train.csv')

In [5]:
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [6]:
train.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [7]:
train = train.drop(['Cabin', 'Ticket', 'Name','Fare','SibSp', 'Parch', 'Age'], axis = 1)

In [8]:
train = train.dropna()

In [9]:
train

,PassengerId,Survived,Pclass,Sex,Embarked
0,1,0,3,male,S
1,2,1,1,female,C
2,3,1,3,female,S
3,4,1,1,female,S
4,5,0,3,male,S
...,...,...,...,...,...
886,887,0,2,male,S
887,888,1,1,female,S
888,889,0,3,female,S
889,890,1,1,male,C


In [10]:
train = pd.get_dummies(train)

In [11]:
train = train.drop('Sex_female', axis = 1)

In [12]:
X_train = train.drop(['Survived', 'PassengerId'], axis = 1)
Y_train = train.Survived
X_train

,Pclass,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,3,1,0,0,1
1,1,0,1,0,0
2,3,0,0,0,1
3,1,0,0,0,1
4,3,1,0,0,1
...,...,...,...,...,...
886,2,1,0,0,1
887,1,0,0,0,1
888,3,0,0,0,1
889,1,1,1,0,0


In [13]:
from sklearn.model_selection import GridSearchCV
from sklearn import tree

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [14]:
clf = tree.DecisionTreeClassifier()
clf

DecisionTreeClassifier()

In [15]:
params = {'criterion':['gini','entropy'], 'max_depth': range(1,30)}

In [16]:
grid_search_cv_clf = GridSearchCV(clf, params, cv = 5)

In [17]:
grid_search_cv_clf.fit(X_train, Y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': range(1, 30)})

In [18]:
grid_search_cv_clf.best_params_

{'criterion': 'gini', 'max_depth': 3}

In [19]:
best_clf = grid_search_cv_clf.best_estimator_

In [20]:
best_clf.fit(X_train, Y_train)

DecisionTreeClassifier(max_depth=3)

In [21]:
best_clf.score(X_train, Y_train)

0.8110236220472441

In [22]:
test = pd.read_csv('/kaggle/input/titanic/test.csv')
test.shape

(418, 11)

In [23]:
test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [24]:
test = test.drop(['Cabin', 'Ticket', 'Name','Fare','SibSp', 'Parch', 'Age'], axis = 1)
test = test.dropna()
test = pd.get_dummies(test)
test = test.drop('Sex_female', axis = 1)
PassengerId = test.PassengerId
X_test = test.drop('PassengerId', axis = 1)
X_test
PassengerId

0       892
1       893
2       894
3       895
4       896
       ... 
413    1305
414    1306
415    1307
416    1308
417    1309
Name: PassengerId, Length: 418, dtype: int64

In [25]:
y_pred = best_clf.predict(X_test)

In [26]:
y_pred = pd.Series(y_pred)

In [27]:
df = pd.DataFrame({'PassengerId': PassengerId, 'Survived':y_pred})

In [28]:
df

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [29]:
df.to_csv('submission.csv', index=False)